In [1]:
import os

In [2]:
%pwd

'e:\\Retail-Analysis-with-Walmart\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Retail-Analysis-with-Walmart'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    required_columns: list


In [7]:
from real_walmart.constants import *
from real_walmart.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.status_file,
            required_columns=config.required_columns,
        )

        return data_validation_config


In [9]:
import pandas as pd
from real_walmart.logging import logger


In [10]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_files_exist(self) -> bool:
        try:
            validation_status = None

            # Read dataset
            df = pd.read_csv("artifacts/data_ingestion/Walmart_Store_sales.csv")
            all_columns = list(df.columns)

            for col in self.config.required_columns:
                if col not in all_columns:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                    return validation_status

            validation_status = True
            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation status: {validation_status}")

            return validation_status

        except Exception as e:
            raise e


In [11]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()

    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_files_exist()

    logger.info("Data validation completed successfully")

except Exception as e:
    raise e


[2025-12-20 08:25:02,009: INFO: common: YAML file loaded successfully: config\config.yaml]
[2025-12-20 08:25:02,014: INFO: common: YAML file loaded successfully: params.yaml]
[2025-12-20 08:25:02,020: INFO: common: Created directory at: artifacts]
[2025-12-20 08:25:02,027: INFO: common: Created directory at: artifacts/data_validation]


[2025-12-20 08:25:02,116: INFO: 2778538828: Data validation completed successfully]
